## Collaborative/Personalized Learning

In [4]:
%matplotlib inline
import sys
import os
helpers_path = os.path.abspath('../')
if helpers_path not in sys.path:
    sys.path.append(helpers_path)

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import Kernel

from online_kernels.klms import KLMS
from online_kernels.knlms import KNLMS
from online_kernels.krls import KRLS
from online_kernels.klmsRFF import klms_rff
from online_kernels.knlmsRFF import knlms_rff
from online_kernels.krlsRFF import krls_rff

from data.datasets import load_dodd
from sklearn.utils import shuffle

In [7]:
iteration = 10
# K individuals (personalized learning systems)
K = 12
# L Canonical models (Edge)
L = 3
# Kernel Dictionary size
D = 50
num_experiments = 1
C = np.zeros((K,N))
c = np.zeros((N,1))
A = np.zeros((K,D))
a = np.zeros((D,1))
u,d,d_true = load_dodd(num_data)

W = (1/sigma) * np.random.normal(size=(2,D))
b = np.random.uniform(0,2*np.pi,(D,1))
h = np.sqrt(2/D) * np.cos(W.T @ u.T + b)
for i in tqdm(range(1,iteration)):
    v = np.random.randint(len(d))
    edge = np.random.randint(K)

    # Learning A gived fixed Cs (Least Squares)

    # Learn C given the A are fixed (Difficult)
    # This is an iteration process (nonconvex)

    # Convert c to C
    
    # Calculate a
    
    # Convert a to A
    
    # Calculate C

100%|██████████| 10/10 [00:00<00:00, 21366.81it/s]

0
1
2
3
4
5
6
7
8
9
